In [ ]:
import os
from dotenv import load_dotenv
import psycopg2
import requests
from typing import Dict, List
import logging 
# Load the environment variables from .env file
load_dotenv('/home/wjones/CC/Capstone/tbd2/LACCTiC/.env', override=True)
logging.basicConfig(filename='db_insert.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s', level=logging.INFO)

# Get the database credentials from environment variables
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASS')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')

# Connect to the database
db_params = {
    'dbname': db_name,
    'user': db_user,
    'password' : db_password,
    'host' : db_host,
    'port' : db_port
}

In [ ]:
def connect_db(db_params):
    """Connect to the PostgreSQL database server."""
    conn = psycopg2.connect(**db_params)
    return conn

In [ ]:
def time_to_seconds(time_str):
    if time_str == 'DNF' or not time_str.replace('.', '', 1).replace(':', '', 1).isdigit():
        return None
    min, sec = time_str.split(':')
    sec, millisec = sec.split('.')
    return int(min) * 60 + int(sec) + float(millisec) / 100

In [ ]:

# this gets the laccttic_id using the TFFRS runner_id
def get_lacctic_id(conn, tffrs_id):
    with conn.cursor() as cur:
        cur.execute("SELECT lacctic_id FROM runners WHERE tffrs_id = %s;", (tffrs_id,))
        result = cur.fetchone()
        return result[0] if result else None

# this gets the primary key using the TFFRS section_id
def get_meet_id(conn, tffrs_meet_id):
    with conn.cursor() as cur:
        cur.execute("SELECT meet_id FROM races WHERE tffrs_meet_id = %s;", (str(tffrs_meet_id),))
        result = cur.fetchone()
        return result[0] if result else None
    
def insert_race_result(conn, race_data):
    with conn.cursor() as cur:
        cur.execute("""
            INSERT INTO raceResults (meet_id, lacctic_id, result, time)
            VALUES (%s, %s, %s, %s)
            ON CONFLICT (meet_id, lacctic_id)
            DO UPDATE SET result = EXCLUDED.result, time = EXCLUDED.time;
        """, (race_data['meet_id'], race_data['lacctic_id'], race_data['result'], race_data['time'],))
        conn.commit()

In [ ]:
import csv
conn = connect_db(db_params)
# Open the output CSV file
#with open('CSV/output.csv', mode='w') as output_file:
    #fieldnames = ['meet_id', 'lacctic_id', 'result', 'time']
   #writer = csv.DictWriter(output_file, fieldnames=fieldnames)

    # Write the header row
    #writer.writeheader()

with open('results.csv', mode='r') as input_file:
    reader = csv.reader(input_file)
    next(reader)  # Skip the header row
    for row in reader:
        tffrs_id = None
        lacctic_id = None
        meet_id, tffrs_url, result, time_str = row
        if 'tfrrs' in tffrs_url and 'athletes' in tffrs_url:
            tffrs_parts = tffrs_url.split('/')
            tffrs_id = tffrs_parts[4]  # The TFFRS ID is expected to be the fifth element
        time_in_seconds = time_to_seconds(time_str)
        if tffrs_id:
            lacctic_id = get_lacctic_id(conn, tffrs_id)
        
        if lacctic_id is not None:
            race_data = {
                'meet_id': get_meet_id(conn,meet_id),
                'lacctic_id': lacctic_id,
                'result': result,
                'time': time_in_seconds
            }
            #print(race_data)

            # Write the race_data dictionary to the CSV file
            #writer.writerow(race_data)

            insert_race_result(conn, race_data)
        else:
            #print(f"No runner found for tffrs_id: {tffrs_id}")
            logging.info(f"No runner found for tffrs_id: {tffrs_id}")